# Modelling Credit Risk
## Introduction
   In this project I will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's credit risk. I'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

### Lending Club Process
Each borrower fills out an application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score and past historical data to assign an interest rate to the borrower. A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. Each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiveing a return on their investments. On the Lending Club marketplace, qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. When they want to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges. The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition the requested amount. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.
### Project Scope
While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this project, I'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, I'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

## Problem Statement
Can I build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

## Data
Lending Club releases data for all of the approved and declined loan applications periodically on [their website](https://www.lendingclub.com/info/statistics.action). The data dictionary can be found [here](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit).

The `LoanStats` sheet describes the approved loans datasets and the `RejectStats` describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, I'll be focusing on data on approved loans only. The approved loans datasets contain information on current loans, completed loans, and defaulted loans.

I'll be focusing on approved loan data from 2007-2011 since most of the loans during this period have already been resolved. More recent datasets contain too many loans that are still in the process of being paid off.

First, I'll need to explore the data to determine which features I want to use and which column represents the target I want to predict.

In [1]:
import pandas as pd
import numpy as np

loans = pd.read_csv('loans_2007.csv')
loans.head(2)

c:\users\deand\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [2]:
loans.shape

(42538, 52)

My first concern from looking at the data is how many columns there are; I will definitely need to drop unnecessary columns to make the data more manageable. I also need to pay attention to data leakage that could cause my model to overfit. Data leakage would be any columns that indicate the loan has been funded.

### Data Cleaning
I will start by removing columns that are either unnecessary or that cause data leakage. Since there are so many columns, I'll have to break them into sections to drop.

First section:

|name	|dtype	|first value	|description|
|---|---|---|---|
id	|object|	1077501	|A unique LC assigned ID for the loan listing.
member_id|	float64|	1.2966e+06|	A unique LC assigned Id for the borrower member.
loan_amnt|	float64|	5000|	The listed amount of the loan applied for by the borrower.
funded_amnt|	float64|	5000|	The total amount committed to that loan at that point in time.
funded_amnt_inv|	float64|	49750|	The total amount committed by investors for that loan at that point in time.
term|	object|	36 months|	The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate|	object|	10.65%|	Interest Rate on the loan
installment|	float64|	162.87|	The monthly payment owed by the borrower if the loan originates.
grade|	object|	B|	LC assigned loan grade
sub_grade|	object|	B2|	LC assigned loan subgrade
emp_title|	object|	NaN|	The job title supplied by the Borrower when applying for the loan.
emp_length|	object|	10+ years|	Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership|	object|	RENT|	The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.
annual_inc|	float64|	24000|	The self-reported annual income provided by the borrower during registration.
verification_status|	object|	Verified|	Indicates if income was verified by LC, not verified, or if the income source was verified
issue_d|	object|	Dec-2011|	The month which the loan was funded
loan_status|	object|	Charged Off|	Current status of the loan
pymnt_plan|	object|	n|	Indicates if a payment plan has been put in place for the loan
purpose|	object|	car|	A category provided by the borrower for the loan request.



In [3]:
cols = ['id','member_id','funded_amnt','funded_amnt_inv','grade',
        'sub_grade','emp_title','issue_d']
loans = loans.drop(cols, axis=1)
loans.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

Using the same method and criteria for dropping columns, I will continue to drop the rest of the necessary columns.

In [4]:
cols = ['zip_code','out_prncp','out_prncp_inv','total_pymnt',
        'total_pymnt_inv','total_rec_prncp','total_rec_int',
       'total_rec_late_fee','recoveries','collection_recovery_fee',
       'last_pymnt_d','last_pymnt_amnt']
loans = loans.drop(cols, axis=1)
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


#### Target Column
I will use the `loan_status` as my target to model since it is the only column that indicates whether a loan was paid off on time. However, since the data in this column is string type, I'll have to convert it to numeric. First I'll explore the values in the column so I can decide the best way to convert the values.

In [5]:
loans['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

Here is the description for the entries above:

|Loan Status|	Count|	Meaning|
|-----------|--------|---------|
Fully Paid	|33136	|Loan has been fully paid off.
Charged Off	|5634|	Loan for which there is no longer a reasonable expectation of further payments.
Does not meet the credit policy. Status:Fully Paid	|1988|	While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
Does not meet the credit policy. Status:Charged Off	|761|	While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
In Grace Period	|20|	The loan is past due but still in the grace period of 15 days.
Late (16-30 days)	|8|	Loan hasn't been paid in 16 to 30 days (late on the current payment).
Late (31-120 days)	|24|	Loan hasn't been paid in 31 to 120 days (late on the current payment).
Current	|961|	Loan is up to date on current payments.
Default	|3|	Loan is defaulted on and no payment has been made for more than 121 days.

Since I am taking the perspective of an investor, I am interested in trying to predict which loan will be paid off on time and which ones will not. Only the 'Fully Paid' and 'Charged Off' values describe the final outcome of the loan. The other values describe loans that are still on going and unknown if the borrower will pay back the loan on time or not. While the 'Default' status resembles the 'Charged Off' status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance.

#### Cleaning the Target Column
Since my main interest is in the whether a loan will be paid off or not, I am only concerned with the 'Fully Paid' and 'Charged Off' status.  I will use binary classification to transform 'Fully Paid' to `1` and 'Charged Off' to `0`. However, I also want to keep in mind the issue of class imbalance, which results in the model favoring either the positive or negative class.

To clean the target column I will drop any rows where `loan_status` is not `Fully Paid` or `Charged Off`. Then I will replace these two values with `1` or `0`.

In [10]:
#dropping rows
criteria = loans[(loans['loan_status'] != 'Fully Paid') & 
                 (loans['loan_status'] != 'Charged Off')].index 
loans = loans.drop(criteria)
#replace values
replace = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0
    }
}
loans = loans.replace(replace)

#### Removing Single Value Columns
Columns in the dataset with only one unique value won't be useful for the model since they don't add any information to the loan applications.

In [11]:
#find the number of unique values in each column
drop_columns = []

for col in loans.columns:
    loans[col].dropna(inplace=True)
    unique = loans[col].unique()
    num_unique = len(unique)
    
    if num_unique == 1:
        drop_columns.append(col)
    else:
        pass
    
loans = loans.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


It looks like I was able to remove nine more columns from the dataframe. Now that I'm finished with cleaning the data I will export the cleaned dataframe as a csv for backup purposes.

In [12]:
loans.to_csv(r'loans_clean.csv',index=False)

## Preparing the Features
Now that the dataset has been cleaned, I can start preparing the data for machine learning by:

* handling missing values
* converting categorical types to numerical
* remove any other unnecessary columns I encounter

Resolving these issues is important because the mathematics underlying most machine learning models assume the data is numerical and contains no missing values.

### Handling Missing Values

In [18]:
null_counts = loans.isnull().sum()
null_counts, loans.shape[0]

(loan_amnt                  0
 term                       0
 int_rate                   0
 installment                0
 emp_length              1036
 home_ownership             0
 annual_inc                 0
 verification_status        0
 loan_status                0
 purpose                    0
 title                     11
 addr_state                 0
 dti                        0
 delinq_2yrs                0
 earliest_cr_line           0
 inq_last_6mths             0
 open_acc                   0
 pub_rec                    0
 revol_bal                  0
 revol_util                50
 total_acc                  0
 last_credit_pull_d         2
 pub_rec_bankruptcies     697
 dtype: int64, 38770)

It appears that most columns in the dataframe do not contain any missing values. However there are 5 columns that do, 2 of which contain a large amount.